In [ ]:
import math
import shutil
import torch
sys.path.append("/home/jyzhao/python_env/gpytorch_local/gpytorch")
sys.path.append("/home/jyzhao/python_env/gpytorch_local/linear_operator")
import gpytorch
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import os
import h5py
import sys
sys.path.append("/resnick/groups/enceladus/jyzhao/Scalable_GPs_jz/regression_gpytorch/src")
from training_utils.GPModel import MultiDeviceExactGPModel
from training_utils.utils import set_seed, read_hdf5, predict_dataset
from training_utils.LBFGS import FullBatchLBFGS
import time
from torch.utils.data import TensorDataset, DataLoader
import argparse

In [ ]:
def plot_training_loss(training_loss, model, save_dir):
    fig, axes = plt.subplots(1,3,figsize=(16, 4))
    for label, color in zip(['source', 'site', 'path'], ['blue', 'red', 'green']):
        if label in model.kernels:
            axes[0].plot(getattr(model, f"{label}_len_training"), label=f"{label} length scale", c = color)
    for label, color in zip(['source', 'site', 'path'], ['blue', 'red', 'green']):
        if label in model.kernels:
            axes[1].plot(getattr(model, f"{label}_var_training"), '--', label=f"{label} variance", c = color)
    if 'eta' in model.kernels:
        axes[1].plot(model.eta_training, ':', label='Between event variance')

    axes[0].set_xlabel('Iteration')
    axes[0].set_ylabel('Hyperparameter Value')
    axes[0].legend()
    axes[1].plot(model.likelihood_noise, ':', label='likelihood noise')
    axes[1].set_xlabel('Iteration')
    axes[1].set_ylabel('Hyperparameter Value')
    axes[1].legend()

    axes[2].set_ylim([0,10])
    axes[2].plot(training_loss, label = "ELBO loss")
    axes[2].set_xlabel('Iteration')
    axes[2].set_ylabel('Training loss');
    axes[2].legend()
    
    fig.savefig(os.path.join(save_dir, 'training_loss_parameters.png'), dpi=300)
    plt.close(fig)
    return

In [ ]:
random_seed = 62
num_training_eqs = 100
num_testing_eqs = 4179
num_training_iter = 2000
set_seed(random_seed)
output_dir = "/resnick/groups/enceladus/jyzhao/Scalable_GPs_jz/regression_gpytorch/output"
trained_model_dir = os.path.join(output_dir, 'trained_exact_models', f'{num_training_eqs}_training_eqs_{num_training_iter}_iters')
if os.path.exists(trained_model_dir):
    shutil.rmtree(trained_model_dir)
os.makedirs(trained_model_dir, exist_ok=True)

In [ ]:
# Load training data
train_data_file = os.path.join(output_dir,'formatted_data', 
                               f'training_{num_training_eqs}_eqs', 
                               f'training_sites_training_eqs_2.00_ASK14_{num_training_eqs}_eqs_1_var_per_eq.h5')
train_x, train_y = read_hdf5(train_data_file)
print(f"Number of training samples: {train_x.shape[0]}")
print(f"Number of training features: {train_x.shape[1]}")
# Centeralize y:
train_y_mean = np.mean(train_y)
train_y = train_y - train_y_mean
train_x = torch.from_numpy(train_x)
train_y = torch.from_numpy(train_y)
train_x, train_y = train_x.contiguous(), train_y.contiguous()

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    n_devices = torch.cuda.device_count()
    print('Planning to run on {} GPUs.'.format(n_devices))
    output_device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    output_device = torch.device("cpu")
    print("Using CPU for training")
train_x = train_x.to(output_device)
train_y = train_y.to(output_device)

In [ ]:
# These are kernel normalizers that are estimated by calculating the average
# kernel distance from 100 training earthquakes
source_normalizer = 92.8112
site_normalizer = 40.3403
path_normalizer = 76.3133
print(f"Kernel lengthscale normalizing constants: source {source_normalizer}, site {site_normalizer}, path {path_normalizer}")

In [ ]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = MultiDeviceExactGPModel(train_x, train_y, likelihood, n_devices, 
                    output_device,
                    source_normalizer=source_normalizer, 
                    site_normalizer=site_normalizer, 
                    path_normalizer=path_normalizer)
model.train()
likelihood.train()
optimizer = FullBatchLBFGS(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [ ]:
# Set initial scale for the effects based on observations in preliminary runs
if 'path' in model.kernels:
    var_init = 0.1
    lengthscale = 1.2
    kernel_ind = model.kernels.index('path')
    model.covar_module.base_kernel.kernels[kernel_ind].outputscale = var_init
    model.covar_module.base_kernel.kernels[kernel_ind].base_kernel.lengthscale = lengthscale
    print(f"Path kernel initial variance is set to {var_init}, lengthscale is set to {lengthscale}")
if 'source' in model.kernels:
    var_init = 0.1
    lengthscale = 0.15
    kernel_ind = model.kernels.index('source')
    model.covar_module.base_kernel.kernels[kernel_ind].outputscale = var_init
    model.covar_module.base_kernel.kernels[kernel_ind].base_kernel.lengthscale = lengthscale
    print(f"Source kernel initial variance is set to {var_init}, lengthscale is set to {lengthscale}")
if 'site' in model.kernels:
    var_init = 0.1
    lengthscale = 0.3
    kernel_ind = model.kernels.index('site')
    model.covar_module.base_kernel.kernels[kernel_ind].outputscale = var_init
    model.covar_module.base_kernel.kernels[kernel_ind].base_kernel.lengthscale = lengthscale
    print(f"Site kernel initial variance is set to {var_init}, lengthscale is set to {lengthscale}")
kernel_ind = model.kernels.index('eta')
model.covar_module.base_kernel.kernels[kernel_ind].outputscale = 0.1
likelihood.noise = 0.1

In [ ]:
training_loss = []
# Initialize model saving directory and save cpu state
training_loss_file = os.path.join(trained_model_dir, 'training_hyperparameter_loss.npy')
model_save_path_pre = os.path.join(trained_model_dir, 'cpu_model_initial.pth')
model_state = model.state_dict()
torch.save(model_state, model_save_path_pre)
likelihood_save_path_pre = os.path.join(trained_model_dir, 'cpu_likelihood_initial.pth')
likelihood_state = likelihood.state_dict()
torch.save(likelihood_state, likelihood_save_path_pre)

In [ ]:
# Move to device
model = model.to(output_device)
likelihood = likelihood.to(output_device)
# Save the gpu model
if device == torch.device("cuda"):
    model_save_path_pre = os.path.join(trained_model_dir, 'gpu_model_initial.pth')
    model_state = model.state_dict()
    torch.save(model_state, model_save_path_pre)
    likelihood_save_path_pre = os.path.join(trained_model_dir, 'gpu_likelihood_initial.pth')
    likelihood_state = likelihood.state_dict()
    torch.save(likelihood_state, likelihood_save_path_pre)
    model_save_path_post_gpu = os.path.join(trained_model_dir, 'gpu_model_trained.pth')
    likelihood_save_path_post_gpu = os.path.join(trained_model_dir, 'gpu_likelihood_trained.pth')
model_save_path_post_cpu = os.path.join(trained_model_dir, 'cpu_model_trained.pth')
likelihood_save_path_post_cpu = os.path.join(trained_model_dir, 'cpu_likelihood_trained.pth')

In [ ]:
preconditioner_size=15
model.train()
likelihood.train()
smallest_loss = np.inf
with gpytorch.settings.max_preconditioner_size(preconditioner_size):
    def closure():
        optimizer.zero_grad()
        output = model(train_x)
        loss = -mll(output, train_y)
        return loss
    loss = closure()
    loss.backward(retain_graph=True)
    training_tqdm = tqdm(range(num_training_iter), leave = True)
    for i in training_tqdm:
        options = {'closure': closure, 'current_loss': loss, 'max_ls': 10,
                   'retain_graph': True}
        loss, _, _, _, _, _, _, fail = optimizer.step(options)
        training_tqdm.set_postfix(loss=f"{loss.item():.6f}",
                                  noise=f"{likelihood.noise.item():.6f}")
        # source event
        if 'source' in model.kernels:
            kernel_idx = model.kernels.index('source')
            model.source_len_training.append(model.covar_module.kernels[kernel_idx].base_kernel.lengthscale.item())
            model.source_var_training.append(model.covar_module.kernels[kernel_idx].outputscale.item())
        # site event
        if 'site' in model.kernels:
            kernel_idx = model.kernels.index('site')
            model.site_len_training.append(model.covar_module.kernels[kernel_idx].base_kernel.lengthscale.item())
            model.site_var_training.append(model.covar_module.kernels[kernel_idx].outputscale.item())
        # path event
        if 'path' in model.kernels:
            kernel_idx = model.kernels.index('path')
            model.path_len_training.append(model.covar_module.kernels[kernel_idx].base_kernel.lengthscale.item())
            model.path_var_training.append(model.covar_module.kernels[kernel_idx].outputscale.item())
        # within event
        if 'eta' in model.kernels:
            kernel_idx = model.kernels.index('eta')
            model.eta_training.append(model.covar_module.kernels[kernel_idx].outputscale.item())
            model.likelihood_noise.append(likelihood.noise.item())

        training_loss.append(loss.item())

        if loss.item() <= smallest_loss:
            smallest_loss = loss.item()
            if output_device == torch.device("cuda"):
                model_save_path_post = model_save_path_post_gpu
                likelihood_save_path_post = likelihood_save_path_post_gpu
            else:
                model_save_path_post = model_save_path_post_cpu
                likelihood_save_path_post = likelihood_save_path_post_cpu
            best_epoch = i

        if fail:
            print('Convergence reached')
            break


In [ ]:
if device == torch.device("cuda"):
    model_save_path_converged = os.path.join(trained_model_dir, 'gpu_model_converged.pth')
    model_state = model.state_dict()
    torch.save(model_state, model_save_path_converged)
    likelihood_save_path_converged = os.path.join(trained_model_dir, 'gpu_likelihood_converged.pth')
    likelihood_state = likelihood.state_dict()
    torch.save(likelihood_state, likelihood_save_path_converged)
else:
    model_save_path_converged = os.path.join(trained_model_dir, 'cpu_model_converged.pth')
    model_state = model.state_dict()
    torch.save(model_state, model_save_path_converged)
    likelihood_save_path_converged = os.path.join(trained_model_dir, 'cpu_likelihood_converged.pth')
    likelihood_state = likelihood.state_dict()
    torch.save(likelihood_state, likelihood_save_path_converged)

In [ ]:
print("Lowest loss achieved at epoch:", best_epoch)
loss_array = np.array([
    model.source_var_training,
    model.source_len_training,
    model.site_var_training,
    model.site_len_training,
    model.path_var_training,
    model.path_len_training,
    model.eta_training,
    model.likelihood_noise,
    training_loss]).T
np.save(training_loss_file, loss_array)

In [ ]:
# Plot the evolution of the training loss and hyperparameters
plot_training_loss(training_loss, model, trained_model_dir)

In [ ]:
# Calculate the model's performance
model.eval()
likelihood.eval()

batch_size = 1024

# Predict on the training set
start = time.time()
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=False)
train_mean, train_lower, train_upper = predict_dataset(train_loader, model, 
                                                       likelihood)
print(f"Time used for train set prediction: {time.time() - start :.1f} s")
train_x,train_y = train_x.cpu(), train_y.cpu()
train_mean, train_lower, test_upper = train_upper.cpu(), train_lower.cpu(), train_upper.cpu()
RMSE_train = np.sqrt(np.mean((train_y.numpy() - train_mean.numpy())**2))
print(f"Training sites Training earthquakes RMSE error {RMSE_train:.3f}; Data mean: {train_y.numpy().mean():.3f}; Data Std: {np.std(train_y.numpy()):.3f}")
if device == torch.device("cuda"):
        torch.cuda.empty_cache()   

In [ ]:
# Get the test data
test_data_file = os.path.join(output_dir,'formatted_data', 
                               f'testing_{num_testing_eqs}_eqs', 
                               f'training_sites_testing_eqs_2.00_ASK14_{num_testing_eqs}_eqs_1_var_per_eq.h5')
test_x, test_y = read_hdf5(test_data_file)
print(f"Number of testing samples: {test_x.shape[0]}")
print(f"Number of testing features: {test_x.shape[1]}")
# Centeralize y:
test_y = test_y - train_y_mean
test_x = torch.from_numpy(test_x).to(device)
test_y = torch.from_numpy(test_y).to(device)
test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)
# Predict on the test set
start = time.time()
test_mean, test_lower, test_upper = predict_dataset(test_loader, model, 
                                                    likelihood, contiguous=True)
print(f"Time used for test set prediction: {time.time() - start :.1f} s")
test_x,test_y = test_x.cpu(), test_y.cpu()
test_mean, test_lower, test_upper = test_mean.cpu(), test_lower.cpu(), test_upper.cpu()
RMSE_test = np.sqrt(np.mean((test_y.numpy() - test_mean.numpy())**2))
print(f"Training sites Testing earthquakes RMSE error {RMSE_test:.3f}; Data mean: {test_y.numpy().mean():.3f}; Data Std: {np.std(test_y.numpy()):.3f}")
if device == torch.device("cuda"):
    torch.cuda.empty_cache() 

In [ ]:
# Get the training earthquake test sites data
test_sites_train_eqs_file = os.path.join(output_dir, 'formatted_data',
                                         f'training_{num_training_eqs}_eqs',
                                         f'testing_sites_training_eqs_2.00_ASK14_{num_training_eqs}_eqs_1_var_per_eq.h5')
test_sites_train_x, test_sites_train_y = read_hdf5(test_sites_train_eqs_file)
print(f"Number of training earthquake test sites: {test_sites_train_x.shape[0]}")
print(f"Number of training earthquake test features: {test_sites_train_x.shape[1]}")
# Centeralize y:
test_sites_train_y = test_sites_train_y - train_y_mean
test_sites_train_x = torch.from_numpy(test_sites_train_x).to(device)
test_sites_train_y = torch.from_numpy(test_sites_train_y).to(device)
test_sites_train_dataset = TensorDataset(test_sites_train_x, test_sites_train_y)
test_sites_train_loader = DataLoader(test_sites_train_dataset, batch_size=batch_size, shuffle=False)
# Predict on the training earthquake test sites
start = time.time()
test_sites_train_mean, test_sites_train_lower, test_sites_train_upper = \
    predict_dataset(test_sites_train_loader, model, likelihood)
print(f"Time used for test sites training set prediction: {time.time() - start :.1f} s")
test_sites_train_x,test_sites_train_y = test_sites_train_x.cpu(), test_sites_train_y.cpu()
test_sites_train_mean, test_sites_train_lower, test_sites_train_upper = \
    test_mean.cpu(), test_lower.cpu(), test_upper.cpu()
RMSE_new_sites_train = np.sqrt(np.mean((test_sites_train_y.numpy() - 
                                            test_sites_train_mean.numpy())**2))
print(f"Testing Sites Training earthquakes RMSE error {RMSE_new_sites_train:.3f}; Data mean: {test_sites_train_y.numpy().mean():.3f}; Data Std: {np.std(test_sites_train_y.numpy()):.3f}")